# BC Grid Extrapolation

Testing errors generated by grid extrapolation for extremely cool spot bolometric corrections. A first test of this will be to use a more extensive Phoenix color grid to explore effects that may be missing from MARCS (aka: condensates).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import LinearNDInterpolator

### Setting up Phoenix Grid Interpolation

First, load required color tables (1 optical, 1 NIR).

In [3]:
phx_col_dir = '/Users/grefe950/Projects/starspot/starspot/color/tab/phx/CIFIST15'
opt_table = np.genfromtxt('{0}/colmag.BT-Settl.server.COUSINS.Vega'.format(phx_col_dir), comments='!')
nir_table = np.genfromtxt('{0}/colmag.BT-Settl.server.2MASS.Vega'.format(phx_col_dir), comments='!')

Generate (linear) interpolation surfaces as a function of $\log g$ and $T_{\rm eff}$.

In [5]:
opt_surface = LinearNDInterpolator(opt_table[:, :2], opt_table[:, 4:8])
nir_surface = LinearNDInterpolator(nir_table[:, :2], nir_table[:, 4:7])

### BT-Settl Colorize a Dartmouth Isochrone

Load a standard isochrone, with MARCS colors.

In [6]:
iso = np.genfromtxt('data/dmestar_00120.0myr_z+0.00_a+0.00_marcs.iso')

Compute colors from Phoenix. Color were shown to be compatible [in another note](http://nbviewer.ipython.org/github/gfeiden/Notebook/blob/master/Daily/20150902_phoenix_cifist_bcs.ipynb).

In [7]:
phx_opt_mags = opt_surface(10.0**iso[:, 1], iso[:, 2])
phx_nir_mags = nir_surface(10.0**iso[:, 1], iso[:, 2])

Convert from surface magnitudes to absolute magnitudes.

In [8]:
for i in range(phx_opt_mags.shape[1]):
    phx_opt_mags[:, i] = phx_opt_mags[:, i] - 5.0*np.log10(10**iso[:, 4]*6.956e10/3.086e18) + 5.0

for i in range(phx_nir_mags.shape[1]):
    phx_nir_mags[:, i] = phx_nir_mags[:, i] - 5.0*np.log10(10**iso[:, 4]*6.956e10/3.086e18) + 5.0

Stack colors with stellar properties to form a new isochrone.

In [9]:
phx_iso = np.column_stack((iso[:, :6], phx_opt_mags))  # stack props with BVRI
phx_iso = np.column_stack((phx_iso, phx_nir_mags))     # stack props/BVRI with JHK

### Load Spotted Isochrone(s)

There are two types of spotted isochrones, one with magnitudes and colors with average surface properties, the other has more detailed information about spot temperatures and luminosities.